In [2]:
from google.colab import drive
drive.mount('/content/drive')

#!pip install -e /content/drive/MyDrive/PhD/Study2_review/Published_Codes_Study_2/Production/codes

Mounted at /content/drive


In [3]:
!pip install torch pandas tqdm scikit-learn
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.7 MB/s eta 0:00:00


In [4]:
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import datetime
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv
from torch_geometric.nn import global_mean_pool as gap
from tqdm import tqdm
import numpy as np
import pandas as pd
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import trange

import warnings
warnings.filterwarnings("ignore")

from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.loader import DataLoader
import torch
from tqdm import tqdm
import pandas as pd
import numpy as np
import sqlite3
import json
tqdm.pandas()

import torch
import numpy as np
from torch_geometric.data import InMemoryDataset, Data

import os


In [5]:
# -------------------------------
# Environment Variables & Device
# -------------------------------
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WANDB_DISABLED"] = "true"

import warnings
warnings.simplefilter("ignore", category=UserWarning)

import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# -------------------------------
# Paths
# -------------------------------
BASE_DATA_PATH = '/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data/datasets'
WEIGHTS_PATH = '/content/weights'
BEST_WEIGHTS_PATH = '/content/best_weights'


import os

# Create the 'weights' folder if it doesn't exist
if not os.path.exists(WEIGHTS_PATH):
    os.makedirs(WEIGHTS_PATH)

# Create the 'best_weights' folder if it doesn't exist
if not os.path.exists(BEST_WEIGHTS_PATH):
    os.makedirs(BEST_WEIGHTS_PATH)

# -------------------------------
# Model & GNN Hyperparameters
# -------------------------------
HYPERPARAMS = {
    'batch_size': 32,
    'lr': {'default': 1e-5}, #{'default': 2e-3},
    'weight_decay': 5e-4,
    'num_features': 1024,
    'ghid1': 32,
    'ghid2': 16,
    'ghid3': 8,
    'heads': 8,
    'p': 0.0, #0.1
    'num_class': 2,
    'max_epoch': 300,
    'epoch_stop_th': 15,
    'class_weights': None,    # To be set per dataset in the experimental loop
}




In [6]:
class tweetData(InMemoryDataset):
    def __init__(self,
                 targets,
                 nodes,
                 edges,
                 root='',
                 filename='train',
                 id_col='node',
                 target_col='label',
                 feature_col='embeddings',
                 seed=9898,
                 num_class=2,
                 transform=None,
                 pre_transform=None
            ):
        self.seed = seed
        self.nodes = nodes
        self.edges = edges
        self.target = target_col
        self.targets = targets
        self.num_class = num_class
        self.filename = filename
        self.col = feature_col
        self.id_col = id_col
        super(tweetData, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0],weights_only=False)

    @property
    def processed_file_names(self):
        return [f'{path}/%s_%s.dataset' % (self.filename, self.seed)]

    def process(self):
        data_list = []
        targets = self.targets
        nodes = self.nodes
        edges = self.edges
        for i in tqdm(range(len(targets))):
            row = targets.iloc[i]
            id_str = row[self.id_col]
            label = torch.tensor([[row[self.target]]], dtype=torch.int64)
            y = torch.zeros(len(label), self.num_class).scatter_(1, label, 1)
            redges = edges[edges.root_node == id_str]
            edge_index = torch.tensor([redges.source.tolist(), redges.target.tolist()], dtype=torch.long)
            rnodes = nodes[nodes.root_node == id_str]
            node_attr = rnodes[self.col]
            node_attr = torch.tensor([np.array(t, dtype=np.float32) for t in node_attr], dtype=torch.float)
            data = Data(
                id=id_str,
                x=node_attr,
                y=y,
                edge_index=edge_index,
                node_ids=rnodes.sid.tolist()
            )
            data_list.append(data)

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

class loadTweetData(InMemoryDataset):
    def __init__(
        self,
        root='/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/data/', #changed estella code
        path='../input/test_0m_bert_9898.dataset',
        transform=None,
        pre_transform=None
    ):
        self.path = path
        super(loadTweetData, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0], weights_only=False)

    @property
    def processed_file_names(self):
        return [self.path]

In [7]:
from tqdm import trange  # use trange instead of tqdm when you want loop index


def sort_indices(ls):
    indices = [0]
    ls_len = [len(x) for x in ls]
    n = 0
    for i in range(len(ls)):
        m = ls_len[i]
        indices.append(m + n)
        n += m
    return indices[:-1]

class userClassifier:
    def __init__(
            self,
            model,
            model_name,
            class_weights,
            batch_size=32,
            max_epoch=100,
            epoch_stop_th=5,
            seed=9898,
            input_folder='../input',
            input_file_name='0m_bert',
            weights_folder='weights',
            best_weights_folder='best_weights',
            best_weight_path=None,
            num_warmup_steps = 500,
            lr = {'default': 2e-3},
            device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),

            print_epoch=True
        ):
        self.model = model
        self.model_name = model_name
        self.max_epoch = max_epoch
        self.epoch_stop_th = epoch_stop_th
        self.class_weights = class_weights
        self.seed = seed
        self.device = device
        self.batch_size = batch_size
        self.input_folder = input_folder
        self.input_file_name = input_file_name
        self.best_weight_path = best_weight_path
        self.weights_folder = weights_folder
        self.best_weights_folder = best_weights_folder
        self.print_epoch = print_epoch
        self.lr = lr

        train_loader = self.data(split='train')
        print("Number of batches in training data:", len(train_loader))
        self.num_batches = len(train_loader)
        self.num_training_steps = self.num_batches * self.max_epoch
        print("num_training_steps:", self.num_training_steps)
        #self.num_warmup_steps = self.num_training_steps / 10
        self.num_warmup_steps = 0

        print("num_warmup_steps:", self.num_warmup_steps)


    def initTraining(self):
        '''
        made a small revision to add a progress bar
        '''
        epoch = 0
        self.setOptimizer()
        self.setScheduler()
        is_better_val = True
        is_better_vals, val_loss_vals, weight_files = [], [], []
        best_weight_path = None

        for epoch in trange(self.max_epoch, desc="Training Epochs"):
            train_loss, train_acc = self.train()
            eval_loss, eval_acc, eval_f1 = self.eval(split='val')

            if epoch > 0:
                is_better_val = eval_loss < val_loss_vals[-1]
            if self.print_epoch:
                tqdm.write(f"""Epoch: {epoch}, Train Loss: {train_loss:.4f}, Train ACC: {train_acc:.4f}, Val Loss: {eval_loss:.4f}, Val ACC: {eval_acc:.4f}""")

            is_better_vals.append(is_better_val)
            val_loss_vals.append(eval_loss)

            if self.weights_folder:
                weight_file_path = self.weight_file(epoch=epoch, loss=eval_loss)
                torch.save(self.model.state_dict(), weight_file_path)
                weight_files.append(weight_file_path)

            if len(is_better_vals) > self.epoch_stop_th:
                if all([v is False for v in is_better_vals[-self.epoch_stop_th:]]):
                    tqdm.write(f'Early Stop! Selected Epoch: {epoch-self.epoch_stop_th}')
                    if len(weight_files) > 0:
                        best_weight_path = weight_files[-self.epoch_stop_th]
                    break
            epoch += 1
        if not best_weight_path:
            print('NO Early Stop')
            best_weight_path = weight_files[-1]

        best_weights = torch.load(best_weight_path, map_location=self.device)
        new_path = best_weight_path.replace(self.weights_folder, self.best_weights_folder)

        print(new_path)

        torch.save(best_weights, new_path)
        self.setVar({'best_weight_path': new_path})
        files = os.listdir(self.weights_folder)
        for f in files:
            if '.pt' in f:
                os.remove(f'{self.weights_folder}/{f}')

    def initTesting(self):
        if self.best_weight_path is not None:
            self.model.load_state_dict(torch.load(self.best_weight_path, map_location=self.device))
        test_loss, test_acc, test_f1, test_f1_weighted, test_f1_macro, test_df = self.eval(split='test')
        print(f"""Test ACC: {test_acc:.4f}, Test F1 (Binary): {test_f1:.4f}, Test F1 (Weighted): {test_f1_weighted:.4f}, Test F1 (Macro): {test_f1_macro:.4f}""")
        pe = performance(
            preds=test_df.prediction,
            targets=test_df.label
        )
        ct = pe.crosstab()
        print('------------------------')
        return test_acc, test_f1, test_f1_weighted, test_f1_macro, test_df

    def train(self):
        torch.manual_seed(self.seed)
        torch.cuda.empty_cache()
        train_loss = 0
        predictions, labels = [], []
        loader = self.data(split='train')
        model = self.model.to(self.device)
        model.train()

        for data in loader:
            self.optimizer.zero_grad()
            data = data.to(self.device)
            _, label = torch.max(data.y, dim=1)
            labels += label.tolist()
            targets = data.y.to(self.device)

            outputs = model(data=data)
            loss = self.getLoss(outputs, targets)

            loss.backward()
            self.optimizer.step()
            self.setVar({'optimizer': self.optimizer})

            _, prediction = torch.max(outputs.data, 1)
            predictions += prediction.tolist()

            train_loss += data.num_graphs * loss.item()

        train_loss = train_loss / loader.dataset.len()
        train_acc = performance(
            predictions,
            labels
        ).accuracy
        self.setVar({'model': model})
        return train_loss, train_acc

    def eval(self, split='val'):
        torch.cuda.empty_cache()
        loader = self.data(split=split)
        eval_loss = 0
        ids, predictions, labels = [], [], []
        model = self.model
        model.eval()

        with torch.no_grad():
            for data in loader:
                data = data.to(self.device)
                ids += data.id
                _, label = torch.max(data.y, dim=1)
                labels += label.tolist()
                targets = data.y.to(self.device)
                outputs = model(data=data)
                loss = self.getLoss(outputs, targets)
                _, prediction = torch.max(outputs.data, 1)
                predictions += prediction.tolist()

                eval_loss += data.num_graphs * loss.item()
        eval_loss = eval_loss / loader.dataset.len()
        evals = performance(
            predictions,
            labels
        )
        if split == 'val':
            return eval_loss, evals.accuracy, evals.f1_score()
        elif split == 'test':
            pre_df = pd.DataFrame({
                'tweet_id': ids,
                'label': labels,
                'prediction': predictions,
            })
            return eval_loss, evals.accuracy, evals.f1_score('binary'), evals.f1_score('weighted'), evals.f1_score('macro'), pre_df

    def getLoss(self, pred, target, binary_label=None, binary=False):
        if binary_label is not None or binary is True:
            binary = True
        cr = self.criterion().to(self.device)
        _, labels = torch.max(target, 1)
        loss = cr(pred, labels)
        return loss

    def criterion(self):
        if self.class_weights is None:
            return torch.nn.CrossEntropyLoss()
        else:
            return torch.nn.CrossEntropyLoss(weight=torch.tensor(self.class_weights))

    def setOptimizer(self):
        optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=self.lr.get('default') if self.lr.get('default') else 2e-3,
            weight_decay=5e-4
        )
        self.setVar({'optimizer': optimizer})

    def setScheduler(self):
        scheduler = get_linear_schedule_with_warmup(
            self.optimizer,
            num_warmup_steps=self.num_warmup_steps,
            num_training_steps=self.num_training_steps
        )
        self.setVar({'scheduler': scheduler})

    def readData(self, split='train'):
        #filepath = f'{self.input_folder}/{split}_{self.input_file_name}_{self.seed}.dataset'
        filepath = f'{self.input_folder}/{split}_{self.input_file_name}_2.dataset'

        data = loadTweetData(path=filepath)
        return data

    def data(self, split):
        loader = self.readData(split=split)
        if split != 'train':
            loader = DataLoader(loader, batch_size=self.batch_size, shuffle=True)
        else:
            loader = DataLoader(loader, batch_size=self.batch_size, shuffle=True)
        return loader

    def weight_file(self, epoch, loss, best=False):
        return f'{self.weights_folder}/{self.model_name}_epoch{epoch}_w{loss:.4f}.pt'

    def setVar(self, var):
        for key, value in var.items():
            setattr(self, key, value)

In [8]:
class gnnGAT(nn.Module):
    def __init__(self, num_features, p, hid1, hid2, hid3, heads, num_class, mode='gap'):
        super(gnnGAT, self).__init__()
        self.p = p
        self.drop = nn.Dropout(p)

        self.gatA1 = GATv2Conv(
            in_channels=num_features,
            out_channels=hid1,
            heads=heads,
            flow='source_to_target',
            dropout=p
        )
        self.gatA2 = GATv2Conv(
            in_channels=hid1 * heads,
            out_channels=hid2,
            heads=heads,
            flow='source_to_target',
        )
        self.gatA3 = GATv2Conv(
            in_channels=hid2 * heads,
            out_channels=hid3,
            heads=1,
            flow='source_to_target',
        )
        self.lin = nn.Linear(num_features, hid2)
        self.out = nn.Sequential(
            nn.Dropout(p),
            nn.Linear(hid2 + hid3, num_class)
        )
        self.mode = mode

    def forward(self, data, *args):
        indices = sort_indices(data.node_ids)
        x = data.x
        edge_index = data.edge_index

        # source to target
        x1 = self.gatA1(x, edge_index)
        x1 = F.dropout(F.relu(x1), training=self.training, p=self.p)
        x1 = self.gatA2(x1, edge_index)
        x1 = self.gatA3(x1, edge_index)
        if self.mode == 'gap':
            x1 = gap(x1, data.batch)
        else:
            x1 = x1[indices]
        embs = self.lin(x[indices])
        x1 = torch.cat([F.normalize(x1), F.normalize(embs)], dim=1)
        out = self.out(x1)
        return F.softmax(out)

In [9]:
from sklearn.metrics import classification_report, f1_score, accuracy_score
import pandas as pd
from tabulate import tabulate

class performance:
    def __init__(
        self,
        preds,
        targets
    ):
        self.preds = preds
        self.targets = targets

    def crosstab(self, print_df=True):
        ct = pd.crosstab(self.targets, self.preds).reset_index()
        label_dt = {1: 'Fake', 0: 'Real', 'label': 'News Type'}
        df = pd.DataFrame({'Label': ct.label.map(label_dt)})
        for col in ct.columns:
            if col in [0, 1]:
                if col in ct.columns:
                    df[col] = ct[col]
        df = df.rename(columns=label_dt)
        if print_df:
            ct_print = tabulate(ct, headers="keys", tablefmt="psql", showindex=False)
            ct_print = '\n'.join([f'\t{x}' for x in ct_print.split('\n')])
            print(ct_print)
        return ct

    @property
    def accuracy(self):
        return accuracy_score(self.preds, self.targets)

    def f1_score(self, average='binary'):
        return f1_score(self.preds, self.targets, average=average)

In [10]:
# -------------------------------
# Main Execution: Experiment Loop
# -------------------------------
#for SEED in [9999,42,67,24,278,2,9898,7,76,8,66,777]: 91, 93
'''
3,5,11,13,17,19,23,29,31,37, 41,43,47,53,59,61,71, 73, 79, 83, 89,
'''
SEED = 2

#use primes
for seed2 in [ 97]:
  results = []
  for dataset in ['fibvid', 'ts']:
      HYPERPARAMS['class_weights'] = [2.25565611, 0.64239691] if dataset == 'fibvid' else [0.98397436, 1.01655629]
      for mode in ['ind']: #['ind','gap']
          for input_type in ['text_emo', 'text_only']:
              for te in ['0m', '1m', '15m', '20m', '60m', '90m', '24h', '48h', 'gt_48h']:
                  model_name = f'gat_{mode}'
                  print(model_name, te, input_type)

                  model = gnnGAT(
                      num_features=HYPERPARAMS['num_features'],
                      p=HYPERPARAMS['p'],
                      hid1=HYPERPARAMS['ghid1'],
                      hid2=HYPERPARAMS['ghid2'],
                      hid3=HYPERPARAMS['ghid3'],
                      heads=HYPERPARAMS['heads'],
                      num_class=HYPERPARAMS['num_class'],
                      mode=mode
                  )

                  print('Hyperparameters:', HYPERPARAMS)
                  print(f'{BASE_DATA_PATH}/'+f'{dataset}_{input_type}_{te}_bertlarge_2')
                  print(seed2)


                  bt = userClassifier(
                      model=model,
                      model_name=model_name,
                      batch_size=HYPERPARAMS['batch_size'],
                      seed=seed2,
                      class_weights=HYPERPARAMS['class_weights'],
                      #input_folder='../input',
                      input_folder = f'{BASE_DATA_PATH}',
                      input_file_name=f'{dataset}_{input_type}_{te}_bertlarge', #added to estella code
                      max_epoch=HYPERPARAMS['max_epoch'],
                      epoch_stop_th=HYPERPARAMS['epoch_stop_th'],
                      #weights_folder='weights',
                      weights_folder=f'{WEIGHTS_PATH}',
                      best_weights_folder=f'{BEST_WEIGHTS_PATH}',
                      best_weight_path=None,
                      lr = HYPERPARAMS['lr'],
                      print_epoch=False
                  )
                  bt.setVar(HYPERPARAMS)
                  bt.initTraining()
                  test_acc, test_f1, test_f1_weighted, test_f1_macro, test_df = bt.initTesting()

                  # Create a unique identifier for this experiment's tables
                  experiment_id = f"{model_name}_{dataset}_{input_type}_{te}"
                  con = sqlite3.connect('results.db')
                  # Export the test DataFrame, confusion matrix, and classification report as separate tables
                  display(test_df.head())
                  test_df.to_sql(f"test_results_{experiment_id}", con, index=False, if_exists='replace')

                  print(classification_report(test_df.label, test_df.prediction))


                  results.append({
                      'model': model_name+'_bertlarge', #added this
                      'dataset': dataset,
                      'time_elapsed': te,
                      'accuracy': test_acc,
                      'f1_score_binary': test_f1,
                      'f1_score_weighted': test_f1_weighted,
                      'f1_score_macro': test_f1_macro,
                      'node_features': input_type,
                      'n_layer': 3,
                      'direction': 'source to target (root)',
                      'timestamp': str(datetime.datetime.now())
                  })
  results = pd.DataFrame(results)
  con = sqlite3.connect('results.db')
  results.to_sql('results', con, index=False, if_exists='append')
  print('--------------------------------')

  # Moving the dbs to a folder

  import shutil

  # Define the source and destination paths
  source_path = "/content/results.db"
  destination_folder = "/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC" # Replace with your desired folder

  # Create the destination folder if it doesn't exist
  import os
  os.makedirs(destination_folder, exist_ok=True)

  lr=HYPERPARAMS['lr']
  es=HYPERPARAMS['epoch_stop_th']
  max_epoch=HYPERPARAMS['max_epoch']
  p = HYPERPARAMS['p']

  destination_path = os.path.join(destination_folder, f"results{p}_BERT500_{lr}_{es}_me{max_epoch}_timelock{seed2}.db")

  # Move the file
  shutil.move(source_path, destination_path)


gat_ind 0m text_emo
Hyperparameters: {'batch_size': 32, 'lr': {'default': 1e-05}, 'weight_decay': 0.0005, 'num_features': 1024, 'ghid1': 32, 'ghid2': 16, 'ghid3': 8, 'heads': 8, 'p': 0.0, 'num_class': 2, 'max_epoch': 300, 'epoch_stop_th': 15, 'class_weights': [2.25565611, 0.64239691]}
/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data/datasets/fibvid_text_emo_0m_bertlarge_2
97
Number of batches in training data: 32
num_training_steps: 9600
num_warmup_steps: 0


Training Epochs: 100%|██████████| 300/300 [07:23<00:00,  1.48s/it]


NO Early Stop
/content/best_weights/gat_ind_epoch299_w0.6900.pt
Test ACC: 0.7163, Test F1 (Binary): 0.8282, Test F1 (Weighted): 0.6939, Test F1 (Macro): 0.5074
	+---------+-----+-----+
	|   label |   0 |   1 |
	|---------+-----+-----|
	|       0 |   7 |  23 |
	|       1 |  38 | 147 |
	+---------+-----+-----+
------------------------


,tweet_id,label,prediction
0,1288113129889517578,1,1
1,1287731817395761152,1,1
2,1288966119676616704,1,0
3,1285589979490656256,1,1
4,1288611555530113024,1,1


              precision    recall  f1-score   support

           0       0.16      0.23      0.19        30
           1       0.86      0.79      0.83       185

    accuracy                           0.72       215
   macro avg       0.51      0.51      0.51       215
weighted avg       0.77      0.72      0.74       215

gat_ind 1m text_emo
Hyperparameters: {'batch_size': 32, 'lr': {'default': 1e-05}, 'weight_decay': 0.0005, 'num_features': 1024, 'ghid1': 32, 'ghid2': 16, 'ghid3': 8, 'heads': 8, 'p': 0.0, 'num_class': 2, 'max_epoch': 300, 'epoch_stop_th': 15, 'class_weights': [2.25565611, 0.64239691]}
/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data/datasets/fibvid_text_emo_1m_bertlarge_2
97
Number of batches in training data: 32
num_training_steps: 9600
num_warmup_steps: 0


...

              precision    recall  f1-score   support

           0       0.32      0.79      0.45        19
           1       0.95      0.72      0.82       114

    accuracy                           0.73       133
   macro avg       0.64      0.75      0.64       133
weighted avg       0.86      0.73      0.77       133

gat_ind gt_48h text_only
Hyperparameters: {'batch_size': 32, 'lr': {'default': 1e-05}, 'weight_decay': 0.0005, 'num_features': 1024, 'ghid1': 32, 'ghid2': 16, 'ghid3': 8, 'heads': 8, 'p': 0.0, 'num_class': 2, 'max_epoch': 300, 'epoch_stop_th': 15, 'class_weights': [0.98397436, 1.01655629]}
/content/drive/MyDrive/PhD/Study2_review/FibVid_EY_KC/Production/data/datasets/ts_text_only_gt_48h_bertlarge_2
97
Number of batches in training data: 20
num_training_steps: 6000
num_warmup_steps: 0


Training Epochs: 100%|██████████| 300/300 [30:38<00:00,  6.13s/it]


NO Early Stop
/content/best_weights/gat_ind_epoch299_w0.5252.pt
Test ACC: 0.7594, Test F1 (Binary): 0.8416, Test F1 (Weighted): 0.7260, Test F1 (Macro): 0.6708
	+---------+-----+-----+
	|   label |   0 |   1 |
	|---------+-----+-----|
	|       0 |  16 |   3 |
	|       1 |  29 |  85 |
	+---------+-----+-----+
------------------------


,tweet_id,label,prediction
0,713,1,1
1,456,1,0
2,402,1,1
3,421,1,1
4,1014,1,0


              precision    recall  f1-score   support

           0       0.36      0.84      0.50        19
           1       0.97      0.75      0.84       114

    accuracy                           0.76       133
   macro avg       0.66      0.79      0.67       133
weighted avg       0.88      0.76      0.79       133

--------------------------------
